In [1]:
import os
import pandas as pd
data_table = pd.read_table("/data/projects/IRF4_mm_trn_data/p300_internal_ChIP/tables/p300_interal_perturb_data_table.txt")


In [2]:
#### Taking functions from enhancerpromoter directly in order to custom MYC gene enhancer signal

def uniquify(seq, idfun=None):
    # order preserving
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        # in old Python versions:
        # if seen.has_key(marker)
        # but in new ones:
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

def unParseTable(table, output, sep):
    fh_out = open(output, 'w')
    if len(sep) == 0:
        for i in table:
            fh_out.write(str(i) + '\n')
    else:
        for line in table:
            fh_out.write(sep.join([str(x) for x in line]) + '\n')

    fh_out.close()

def parseTable(fn, sep, header = False,excel = False):
    fh = open(fn)

    line = fh.readline()
    if line.count('\r') >0:
        excel =True
    fh.close()
    fh = open(fn)
    if header == True:
        header = fh.readline() #disposes of the header
    
    #first figure out if this table came from excel and has \r for line breaks

    table = []
    if excel:
        lines_raw = fh.readlines()

        for line in lines_raw:
            line = string.replace(line,'\r','')
            line = line.rstrip()
            table.append(line.split(sep))
    else:
        for line in fh:
            line = line.rstrip().split(sep)
            table.append(line)

    fh.close()

    return table


In [4]:
import os
import pandas as pd
range_start = 106137436
range_end = 106336347
enhancer_promoter_path =  "/data/projects/IRF4_mm_trn_data/p300_internal_ChIP/enhancer_promoter/"
for folder in os.listdir(enhancer_promoter_path ):
    if folder.endswith('_compare'):
        folder_path = os.path.join(enhancer_promoter_path, folder)   
        analysisName = folder
        peakTablePath = enhancer_promoter_path+ folder+"/"+ folder+"_PEAK_TABLE.txt"
        
        # Search for files with the specified extension within the folder
        df = pd.read_table(peakTablePath)
        overlap_condition = (df['CHROM'] == "chr14") & (df['START'].between(range_start, range_end) | df['STOP'].between(range_start, range_end))
        df.loc[overlap_condition,'PROXIMAL_GENES'] = "MYC"
        
        peakTablePath = enhancer_promoter_path+ folder+"/"+ folder+"_with_MYC_PEAK_TABLE.txt"
        df.to_csv(peakTablePath, index=False, sep='\t') 
        peakTable = parseTable(peakTablePath,'\t')

        '''
        takes the peak table and makes a gene centric table

        '''

        geneDict = {}

        geneTable = [['GENE','%s_TSS_SIGNAL' % (analysisName),'%s_DISTAL_SIGNAL' % (analysisName)]]

        #now iterate through the table
        for line in peakTable[1:]:
            regionLength = int(line[4])

            signal = float(line[9]) * regionLength


            #genes where this particular peak overlaps the tss prox window
            #where there are both overlap and proximal meet
            if len(line) == 15:
                overlapGeneList = [gene for gene in line[-2].split(',') if len(gene) > 0]
                if overlapGeneList.count('107'):
                    print(line)
                    sys.exit()
                for overlapGene in overlapGeneList:
                    if geneDict.__contains__(overlapGene) == False:
                        geneDict[overlapGene] = {'tss':0.0,'distal':0.0}
                    #there can be a nasty 1 overlap case where the region might overlap by the overlapping gene list, but not be real
                    if int(line[5]) == 1:
                        geneDict[overlapGene]['tss'] += signal
                    else: #this is the case where the mycn site is just outside of the promoter or overlapping the gene locus/body these are rar
                        geneDict[overlapGene]['distal'] += signal

                proximalGeneList = [gene for gene in line[-1].split(',') if len(gene) > 0]
                for proximalGene in proximalGeneList:
                    if geneDict.__contains__(proximalGene) == False:
                        geneDict[proximalGene] = {'tss':0.0,'distal':0.0}
                    if int(line[5]) == 0:
                        geneDict[proximalGene]['distal'] += signal
            #where there's just overlap
            if len(line) == 14:
                overlapGeneList = [gene for gene in line[-1].split(',') if len(gene) > 0]
                if overlapGeneList.count('107'):
                    print(line)
                    sys.exit()
                for overlapGene in overlapGeneList:
                    if geneDict.__contains__(overlapGene) == False:
                        geneDict[overlapGene] = {'tss':0.0,'distal':0.0}
                    #there can be a nasty 1 overlap case where the region might overlap by the overlapping gene list, but not be real
                    if int(line[5]) == 1:
                        geneDict[overlapGene]['tss'] += signal
                    else: #this is the case where the mycn site is just outside of the promoter or overlapping the gene locus/body these are rar
                        geneDict[overlapGene]['distal'] += signal



        geneList = geneDict.keys()
        geneList = uniquify(geneList)
        geneList.sort()

        for gene in geneList:
            newLine = [gene]
            newLine.append(geneDict[gene]['tss'])
            newLine.append(geneDict[gene]['distal'])
            geneTable.append(newLine)
            
        geneTablePath =  enhancer_promoter_path+ folder+"/"+ folder+"_peaks_with_MYC_GENE_TABLE.txt"
        unParseTable(geneTable,geneTablePath,'\t')

        



